In [ ]:
# !pip install IPython
from IPython.display import display, HTML, Image

In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day05.git #코랩 사용

In [ ]:
Image('/content/LG_time_series_day05/image/intro1.png')

# [머신러닝 기반 시계열 분석 2 실습]
# ANN
## [ANN - Classification(이진 분류, 다중 분류)]

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기

In [ ]:
''' 기본 모듈 및 시각화 모듈 '''
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

''' 데이터 전처리 모듈 '''
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

''' Linear Regression 모듈 '''
from sklearn.linear_model import LinearRegression

''' Neural Network Classifier(분류기) 모듈 '''
from sklearn.neural_network import MLPClassifier

''' 결과 평가용 모듈 '''
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

''' 기타 optional'''
import warnings, itertools
warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns', None) # 모든 열 이름 보기

## 2. 분석데이터: Parkinsons (이진 분류 문제)

<center> <a href='https://archive.ics.uci.edu/ml/datasets/Parkinsons'> https://archive.ics.uci.edu/ml/datasets/Parkinsons </a></center>



### 설명변수 (X)
- name - ASCII subject name and recording number
- MDVP:Fo(Hz) - Average vocal fundamental frequency
- MDVP:Fhi(Hz) - Maximum vocal fundamental frequency
- MDVP:Flo(Hz) - Minimum vocal fundamental frequency
- MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP - Several measures of variation in fundamental frequency
- MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA - Several measures of variation in amplitude
- NHR,HNR - Two measures of ratio of noise to tonal components in the voice
- RPDE,D2 - Two nonlinear dynamical complexity measures
- DFA - Signal fractal scaling exponent
- spread1,spread2,PPE - Three nonlinear measures of fundamental frequency variation

### 반응변수 (Y)
- status - Health status of the subject (one) - Parkinson's, (zero) - healthy


### 데이터 불러오기

In [ ]:
data = pd.read_csv('/content/LG_time_series_day05/data/Parkinsons.csv', index_col=0)
# data = pd.read_csv('./data/Parkinsons.csv', index_col=0) #로컬

## 2-1. 데이터 전처리 및 탐색적 데이터 분석

### 데이터 확인

In [ ]:
print('Data shape: {}'.format(data.shape))
data.head()

### 클래스 비율 확인
0: Healthy <br>
1: Parkinson's Disease

In [ ]:
data.groupby('status')['status'].count()

### 설명변수(X)와 반응변수(Y) 정의

In [ ]:
x = data.drop(labels='status', axis=1)
y = data['status']

In [ ]:
display(x.head())
display(y.head())

### 학습 데이터(Training Dataset)와 테스트 데이터(Testing Dataset) 분리
- 학습데이터와 테스트 데이터의 클래스 비율이 달라지지 않도록 stratify 옵션 사용


In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, stratify=y, test_size=0.3)

### Matplotlib 을 이용한 클래스 비율 plotting
- import matplotlib.pyplot as plt
- plt.figure: plot을 그리기 위해 가장 먼저 선언 & plot 크기 지정
- plt.subplot: plot을 한번에 여러개 그리기 위해 선언(행크기/열크기/열번호or행번호)
- plt.hist: 히스토그램
- plt.plot(x,y): 2차원 
- plt.xlim: x축 (최소,최대) 지정
- plt.ylim: y축 (최소,최대) 지정
- plt.title: 해당 plot의 제목 지정
- plt.show(): plot을 그리기 위해 가장 "마지막" 선언

In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(121)
plt.hist(train_y)
plt.ylim(0, 110)
plt.xticks(np.arange(0, 2, 1))
plt.title('Training Data')

plt.subplot(122)
plt.hist(test_y)
plt.ylim(0, 110)
plt.xticks(np.arange(0, 2, 1))
plt.title('Testing Data')
plt.show()

### 정규화 : Standardization(표준정규화) with Standard Scaler



- 설명변수(X) 내 변수별 평균이 0이 되도록 변환
- Training Data를 기준으로 Scaler를 구성한 뒤, Testing Data에 Scaler를 적용

\begin{align}
z = \frac{(x - \mu)}{\sigma}
\end{align}

In [ ]:
x.describe()

In [ ]:
Image('/content/LG_time_series_day05/image/intro2.png')

##### 참조: https://www.jeremyjordan.me/batch-normalization/

In [ ]:
scaler = StandardScaler()
scaler.fit(train_x)

train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
pd.DataFrame(train_x, columns = x.columns).describe()

## 2-2. 모델링: Multilayer Perceptron (MLP) Classifier

<a href='http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html'> Scikit-Learn MLP Classifier Manual </a>

- Hidden layer 개수: 얼마나 깊은 신경망을 구축할 것인지
- Node 개수: 각 layer(층)마다 y를 구분짓기 좋은 특징을 몇개 부여할 것인지

### MLP Classifier 구조

In [ ]:
Image('/content/LG_time_series_day05/image/intro3.png')

### MLP Classifier 정의
- <b>activation</b> - activation function 타입 (identity, logistic, tanh, <font color='red'>relu</font>) <br>

- <b>batch_size</b> - stochastic optimizer가 사용할 minibatch 크기 <br>
- <b>max_iter  </b> - stochastic optimizer의 최대 iteration 횟수 ( = Epochs )<br>

- <b>alpha     </b> - Learning Rate (과적합 방지용) <br>
- <b>solver    </b> - 경사하강법의 종류 (<font color='red'>adam</font>, sgd, lbfgs) <br>

-- lbfgs: L-BFGS 준-뉴턴 방식 의 최적화 알고리즘으로, 제한된 컴퓨터 메모리를 이용하여 기존 BFGS 알고리즘을 속도면에서 개선한 알고리즘

In [ ]:
clf_mlp = MLPClassifier(activation='relu', alpha=0.0001, batch_size=10,
                        hidden_layer_sizes=(10), max_iter=500,
                        solver='adam', verbose = True, random_state = 2022)

### MLP Classifier 학습

In [ ]:
clf_mlp.fit(train_x, train_y)

### 학습 상태 확인 (learning curve)

In [ ]:
plt.figure(figsize=(20,10))

train_loss_values = clf_mlp.loss_curve_
plt.plot(train_loss_values,label='Train Loss')

plt.legend(fontsize=20)
plt.title("Learning Curve of trained MLP Classifier", fontsize=18)
plt.show()

## 2-3. Multilayer Perceptron 모델 성능 평가

### 학습된 MLP Classifier 결과 확인: Training Data

In [ ]:
Image('/content/LG_time_series_day05/image/intro4.png')

- True: 올바르게 예측한 경우
- False: 잘못 예측한 경우
- Positive: 관심 대상이 되는 클래스, Parkinson's Disease
- Negative: 관심 대상이 아닌 클래스, Healthy

In [ ]:
train_y_pred = clf_mlp.predict(train_x)

In [ ]:
cm_train = confusion_matrix(y_true=train_y, y_pred=train_y_pred)

In [ ]:
plt.figure(figsize=(5, 4))
sns.heatmap(data=cm_train, annot=True, fmt='d', annot_kws={'size': 18}, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### 학습된 MLP Classifier 성능 평가: 다양한 평가 지표


- 다양한 분류 평가 지표를 계산하기 위한 함수 정의

In [ ]:
def validation_measure(cm):
    TN, FP, FN, TP = cm.flatten()
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)
    precision = TP/(TP+FP)
    F1_score = 2/(1/sensitivity + 1/precision)
    return accuracy, sensitivity, specificity, precision, F1_score

In [ ]:
train_acc, train_sens, train_spec, train_prec, train_f1 = validation_measure(cm_train)
print('Train Accuracy   : {:.3f}'.format(train_acc))
print('Train Sensitivity: {:.3f}'.format(train_sens))
print('Train Specificity: {:.3f}'.format(train_spec))
print('Train Precision  : {:.3f}'.format(train_prec))
print('Train F1 Score   : {:.3f}'.format(train_f1))

### 학습된 MLP Classifier 성능 평가: Testing Data

In [ ]:
test_y_pred = clf_mlp.predict(test_x)

cm_test = confusion_matrix(y_true=test_y, y_pred=test_y_pred)

plt.figure(figsize=(5, 4))
sns.heatmap(data=cm_test, annot=True, fmt='d', annot_kws={'size': 18}, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### 학습된 MLP Classifier 성능 평가: 다양한 평가 지표

In [ ]:
test_acc, test_sens, test_spec, test_prec, test_f1 = validation_measure(cm_test)
print('Test Accuracy   : {:.3f}'.format(test_acc))
print('Test Sensitivity: {:.3f}'.format(test_sens))
print('Test Specificity: {:.3f}'.format(test_spec))
print('Test Precision  : {:.3f}'.format(test_prec))
print('Test F1 Score   : {:.3f}'.format(test_f1))

## 3. 분석데이터 : Wine Type (다중 분류 문제)

### 설명변수 (X)
- Alcohol
- Malic acid
- Ash
- Alcalinity of ash
- Magnesium
- Total phenols
- Flavanoids
- Nonflavanoid phenols
- Proanthocyanins
- Color intensity
- Hue
- OD280/OD315 of diluted wines
- Proline

### 반응변수 (Y)
- Wine Type

### 데이터 불러오기

In [ ]:
data = pd.read_csv('/content/LG_time_series_day05/data/Wine.csv')
# data = pd.read_csv('./data/Wine.csv') #로컬

## 3-1. 데이터 전처리 및 탐색적 데이터 분석

### 데이터 확인

In [ ]:
print('Data shape: {}'.format(data.shape))
data.head()

### 클래스 비율 확인

In [ ]:
data.groupby('Wine Type')['Wine Type'].count()

### 설명변수(X)와 반응변수(Y) 정의

In [ ]:
x = data.drop(labels='Wine Type', axis=1)
y = data['Wine Type']

In [ ]:
display(x.head())
display(y.head())

### 학습 데이터(Training Dataset)와 테스트 데이터(Testing Dataset) 분리
- 학습데이터와 테스트 데이터의 클래스 비율이 달라지지 않도록 stratify 옵션 사용

In [ ]:
Image('/content/LG_time_series_day05/image/intro5.png')

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, stratify=y, test_size=0.3)

### Matplotlib 을 이용한 클래스 비율 plotting

In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(121)
plt.hist(train_y)
plt.ylim(0, 60)
plt.xticks(np.arange(1, 4, 1))
plt.title('Training Data')

plt.subplot(122)
plt.hist(test_y)
plt.ylim(0, 60)
plt.xticks(np.arange(1, 4, 1))
plt.title('Testing Data')
plt.show()

### 정규화 : Standardization(표준정규화) with Standard Scaler

In [ ]:
x.describe()

In [ ]:
scaler = StandardScaler()
scaler.fit(train_x)

train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
pd.DataFrame(train_x, columns = x.columns).describe()

## 3-2. 모델링: Multilayer Perceptron (MLP) Classifier

<a href='http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html'> Scikit-Learn MLP Classifier Manual </a>

### MLP Classifier 구조

In [ ]:
Image('/content/LG_time_series_day05/image/intro6.png')

### MLP Classifier 정의

- <b>activation</b> - activation function 타입 (identity, logistic, tanh, <font color='red'>relu</font>) <br>
- <b>batch_size</b> - stochastic optimizer가 사용할 minibatch 크기 <br>
- <b>max_iter  </b> - stochastic optimizer의 최대 iteration 횟수 ( = Epochs )<br>
- <b>alpha     </b> - Learning Rate (과적합 방지용) <br>
- <b>solver    </b> - 경사하강법의 종류 (<font color='red'>adam</font>, sgd, lbfgs) <br>

In [ ]:
clf_mlp = MLPClassifier(activation='relu', alpha=0.0001, batch_size=100,
                        hidden_layer_sizes=(7), max_iter=500,
                        solver='adam', verbose = True, random_state = 2022)

### MLP Classifier 학습

In [ ]:
clf_mlp.fit(train_x, train_y)

### 학습 상태 확인 (learning curve)

In [ ]:
plt.figure(figsize=(20,10))

train_loss_values = clf_mlp.loss_curve_
plt.plot(train_loss_values,label='Train Loss')

plt.legend(fontsize=20)
plt.title("Learning Curve of trained MLP Classifier", fontsize=18)
plt.show()

## 3-3. Multilayer Perceptron 모델 성능 평가

### 학습된 MLP Classifier 결과 확인: Training Data

In [ ]:
train_y_pred = clf_mlp.predict(train_x)

In [ ]:
cm_train = confusion_matrix(y_true=train_y, y_pred=train_y_pred)
cm_train = pd.DataFrame(cm_train, columns=['1', '2', '3'], index=['1', '2', '3'])

In [ ]:
plt.figure(figsize=(5, 4))
sns.heatmap(data=cm_train, annot=True, fmt='d', annot_kws={'size': 18}, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### Training 데이터 예측확률 확인

In [ ]:
train_y_pred_proba = clf_mlp.predict_proba(train_x)
train_y_pred_proba = pd.DataFrame(train_y_pred_proba)
train_y_pred_proba.index = train_y.index.copy()

In [ ]:
train_results = pd.concat([train_y_pred_proba, train_y], axis=1)
train_results.columns = ['Class Probability 1', 'Class Probability 2', 'Class Probability 3', 'Wine Type']
train_results.head()

### 학습된 MLP Classifier 성능 평가: Testing Data

In [ ]:
test_y_pred = clf_mlp.predict(test_x)

cm_test = confusion_matrix(y_true=test_y, y_pred=test_y_pred)
cm_test = pd.DataFrame(cm_test, columns=['1', '2', '3'], index=['1', '2', '3'])

plt.figure(figsize=(5, 4))
sns.heatmap(data=cm_test, annot=True, fmt='d', annot_kws={'size': 18}, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### Testing 데이터 예측확률 확인

In [ ]:
test_y_pred_proba = clf_mlp.predict_proba(test_x)
test_y_pred_proba = pd.DataFrame(test_y_pred_proba)
test_y_pred_proba.index = test_y.index.copy()

In [ ]:
test_results = pd.concat([test_y_pred_proba, test_y], axis=1)
test_results.columns = ['Class Probability 1', 'Class Probability 2', 'Class Probability 3', 'Wine Type']
test_results.head()

In [ ]:
print("정확도 : {:.3f}".format(accuracy_score(test_y,test_y_pred)))
print("오차 행렬 \n",confusion_matrix(test_y,test_y_pred))

print(classification_report(test_y,test_y_pred))